In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

In [ ]:
df_songs = pd.read_csv('song_dataset.csv')
df_songs

,user,song,play_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1,Nothing from Nothing,To Die For,Billy Preston,1974
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1,Under Cold Blue Stars,Under Cold Blue Stars,Josh Rouse,2002
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1,Riot Radio (Soundtrack Version),Nick & Norah's Infinite Playlist - Original Mo...,The Dead 60s,0
...,...,...,...,...,...,...,...
102622,21f4ac98aa1665bd42027ba12184a939ff435f59,SOKAKHH12AF72A5BAF,3,87,Hopeless Romantic,Bouncing Souls,1999
102623,21f4ac98aa1665bd42027ba12184a939ff435f59,SONPOXM12A8C1440C2,4,Space Olympics,Incredibad,The Lonely Island,2009
102624,21f4ac98aa1665bd42027ba12184a939ff435f59,SOPREHY12AB01815F9,8,I'm On A Boat,Incredibad,The Lonely Island / T-Pain,2009
102625,21f4ac98aa1665bd42027ba12184a939ff435f59,SOQXKUV12A6D4FB4C9,3,Amityville,The Marshall Mathers LP,Eminem / Bizarre,2000


In [ ]:
user_select = widgets.Select(
    options=df_songs.user.unique(),
    description='Select User:',
    disabled=False,
    layout=widgets.Layout(width='auto', height='auto'),
)
user_select.style.description_width = 'initial'
display(user_select)

Select(description='Select User:', layout=Layout(height='auto', width='auto'), options=('b80344d063b5ccb3212f7…

In [ ]:
# Create a SelectMultiple widget with song titles as options
song_select = widgets.SelectMultiple(
    options=df_songs[df_songs['user']==user_select.value]['title'].unique(),
    description='Select Songs (Hold CTRL and select multiple):',
    disabled=False,
    layout=widgets.Layout(width='auto', height='auto'),
    multiple=True
)
# Optionally, adjust the font size for better readability
song_select.style.description_width = 'initial'
display(song_select)

SelectMultiple(description='Select Songs (Hold CTRL and select multiple):', layout=Layout(height='auto', width…

In [ ]:
selected_songRecord=df_songs[df_songs['title'].isin(song_select.value)].where(df_songs['user']==user_select.value).dropna()

# selected_songRecord

In [ ]:
# 1. Find the most frequent artist and year from the selected songs:
frequent_artist = (selected_songRecord.groupby('artist_name')['artist_name'].count()* selected_songRecord.groupby('artist_name')['play_count'].sum()).idxmax()
frequent_year = (selected_songRecord.groupby('year')['year'].count()* selected_songRecord.groupby('year')['play_count'].sum()).idxmax()
frequent_release=(selected_songRecord.groupby('release')['release'].count()* selected_songRecord.groupby('release')['play_count'].sum()).idxmax()
flag=0
def weightCalculater():
  w_artist=0
  w_year=0
  w_release=0
  user_records=df_songs[df_songs['user']==user_select.value]
  w_artist=(user_records[user_records['artist_name']==frequent_artist].play_count.sum())/3
  if frequent_year==0:
    w_year=0
    flag=1
  else:
    w_year=(user_records[user_records['year']==frequent_year].play_count.sum())/3

  w_release=(user_records[user_records['release']==frequent_release].play_count.sum())/3
  weight={'artist_name':w_artist,'year':w_year, 'release':w_release}
  weight=sorted(weight.items(), key=lambda x: x[1], reverse=True)
  return weight



def recommendation_engine():
  top_three=pd.DataFrame()
  top_two=pd.DataFrame()
  top_one=pd.DataFrame()
  recommendations=pd.DataFrame()

  weight=weightCalculater()
  df_songsSearch=df_songs[df_songs['user']!=user_select.value]
  feature={}
  for f, v in weight:
    if f == 'artist_name':
      feature[f]=frequent_artist
    if f == 'year':
      feature[f]=frequent_year
    if f == 'release':
      feature[f]=frequent_release

  if flag==1:
    print("fl")
    top_two=df_songsSearch[df_songsSearch[weight[0][0]]==	feature[weight[0][0]]]
    top_one=top_two[top_two[weight[1][0]]==feature[weight[1][0]]]
  elif flag==0:
    top_three=df_songsSearch[df_songsSearch[weight[0][0]]==	feature[weight[0][0]]]
    top_two=top_three[top_three[weight[1][0]]==feature[weight[1][0]]]
    top_one=top_two[top_two[weight[2][0]]==feature[weight[2][0]]]

  recommendations=top_one.copy()
  recommendations.drop_duplicates(subset=['song'], inplace=True)
  if(recommendations.shape[0]<10):
    recommendations=pd.concat([recommendations,top_two],ignore_index=True)
    recommendations.drop_duplicates(subset=['song'], inplace=True)
    if(recommendations.shape[0]<10):
      if flag==0:
        recommendations=pd.concat([recommendations,top_three],ignore_index=True)
      if(recommendations.shape[0]<10):
        recommendations=pd.concat([recommendations,df_songs[df_songs[weight[1][0]]==feature[weight[1][0]]]],ignore_index=True)
        if(recommendations.shape[0]<10):
          recommendations=pd.concat([recommendations,df_songs[df_songs[weight[2][0]]==feature[weight[2][0]]]],ignore_index=True)
          if(flag==0):
            if(recommendations.shape[0]<10):
              recommendations=pd.concat([recommendations,df_songs[df_songs[weight[0][0]]==feature[weight[0][0]]]],ignore_index=True)

  recommendations.drop_duplicates(subset=['song'], inplace=True)
  # recommendations = recommendations[recommendations.user != user_select.value]
  return recommendations
recommendations=recommendation_engine()
recommendations = recommendations.drop(['user', 'play_count'], axis=1)
recommendations.head(5)


,song,title,release,artist_name,year
0,SOBYHAJ12A6701BF1D,Constellations,In Between Dreams,Jack Johnson,2005
1,SOFFJPX12A6D4F7456,Winter In The Hamptons,Nashville,Josh Rouse,2005
2,SOOGGEX12A58A7DACF,Street Lights,Nashville,Josh Rouse,2005
3,SORSAJY12A6D4F7457,Carolina (Album Version),Nashville,Josh Rouse,2005
4,SOVZRXZ12A58A77A88,It's The Night Time,Nashville,Josh Rouse,2005
